In [1]:
import requests
import pandas as pd

In [2]:
url = 'https://www.basketball-reference.com/teams/'
html = requests.get(url).content
df_list = pd.read_html(html)
tames = df_list[0]

    Lg -- League  From -- First year  To -- Last year  G -- Games  W -- Wins  L -- Losses  
    W/L% -- Win-Loss Percentage  Plyfs -- Years team made the playoffs  
    Div -- Years team finished first (or tied for first) in the division  
    Conf -- Years team won the conference championship  Champ -- Years team won the league championship

In [3]:
tames

,Franchise,Lg,From,To,Yrs,G,W,L,W/L%,Plyfs,Div,Conf,Champ
0,Atlanta Hawks,NBA,1950,2019,70,5552,2746,2806,0.495,46,11,0,1
1,Atlanta Hawks,NBA,1969,2019,51,4134,2048,2086,0.495,33,5,0,0
2,St. Louis Hawks,NBA,1956,1968,13,1005,553,452,0.550,12,6,0,1
3,Milwaukee Hawks,NBA,1952,1955,4,281,91,190,0.324,0,0,0,0
4,Tri-Cities Blackhawks,NBA,1950,1951,2,132,54,78,0.409,1,0,0,0
5,Boston Celtics,NBA/BAA,1947,2019,73,5724,3378,2346,0.590,56,31,9,17
6,Brooklyn Nets,NBA/ABA,1968,2019,52,4222,1824,2398,0.432,27,5,2,2
7,Brooklyn Nets,NBA,2013,2019,7,574,242,332,0.422,4,0,0,0
8,New Jersey Nets,NBA,1978,2012,35,2822,1186,1636,0.420,16,4,2,0
9,New York Nets,NBA,1977,1977,1,82,22,60,0.268,0,0,0,0


In [4]:
team_l = list(tames[tames.To==2019].Franchise.unique())

In [5]:
teams_d = {}
for team in team_l:
    name = team.split(' ')
    if len(name) == 2:
        abb = name[0][:3].upper()
    else:
        abb = ''
        for word in name:
            abb += word[0].upper()
    team_name = '_'.join(name)
    teams_d[team_name] = abb

In [6]:
teams_d['Oklahoma_City_Thunder'] = 'OKC'
teams_d['Brooklyn_Nets'] = 'NJN'
teams_d['Charlotte_Hornets'] = 'CHO'
teams_d['Brooklyn_Nets'] = 'BRK'
teams_d['Portland_Trail_Blazers'] = 'POR'
teams_d

{'Atlanta_Hawks': 'ATL',
 'Boston_Celtics': 'BOS',
 'Brooklyn_Nets': 'BRK',
 'Charlotte_Hornets': 'CHO',
 'Chicago_Bulls': 'CHI',
 'Cleveland_Cavaliers': 'CLE',
 'Dallas_Mavericks': 'DAL',
 'Denver_Nuggets': 'DEN',
 'Detroit_Pistons': 'DET',
 'Golden_State_Warriors': 'GSW',
 'Houston_Rockets': 'HOU',
 'Indiana_Pacers': 'IND',
 'Los_Angeles_Clippers': 'LAC',
 'Los_Angeles_Lakers': 'LAL',
 'Memphis_Grizzlies': 'MEM',
 'Miami_Heat': 'MIA',
 'Milwaukee_Bucks': 'MIL',
 'Minnesota_Timberwolves': 'MIN',
 'New_Orleans_Pelicans': 'NOP',
 'New_York_Knicks': 'NYK',
 'Oklahoma_City_Thunder': 'OKC',
 'Orlando_Magic': 'ORL',
 'Philadelphia_76ers': 'PHI',
 'Phoenix_Suns': 'PHO',
 'Portland_Trail_Blazers': 'POR',
 'Sacramento_Kings': 'SAC',
 'San_Antonio_Spurs': 'SAS',
 'Toronto_Raptors': 'TOR',
 'Utah_Jazz': 'UTA',
 'Washington_Wizards': 'WAS'}

In [12]:
url = 'https://www.basketball-reference.com/players/j/jamesle01/gamelog/2004/'
html = requests.get(url).content
df_list = pd.read_html(html)
lebron_james = df_list[-1]

    Rk -- Rank  G -- Season Game  Age -- Age of Player at the start of February 1st of that season.  Tm -- Team
    Opp -- Opponent  GS -- Games Started  MP -- Minutes Played  FG -- Field Goals  FGA -- Field Goal Attempts
    FG% -- Field Goal Percentage  3P -- 3-Point Field Goals  3PA -- 3-Point Field Goal Attempts  
    3P% -- 3-Point Field Goal Percentage  FT -- Free Throws  FTA -- Free Throw Attempts  
    FT% -- Free Throw Percentage  ORB -- Offensive Rebounds  DRB -- Defensive Rebounds  TRB -- Total Rebounds  
    AST -- Assists  STL -- Steals  BLK -- Blocks  TOV -- Turnovers  PF -- Personal Fouls  PTS -- Points  
    GmSc -- Game Score  +/- -- Plus/Minus

In [13]:
lebron_james

,Rk,G,Date,Age,Tm,Unnamed: 5,Opp,Unnamed: 7,GS,MP,...,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,GmSc,+/-
0,1,1,2003-10-29,18-303,CLE,@,SAC,L (-14),1,42:00,...,4,6,9,4,0,2,3,25,24.7,-9
1,2,2,2003-10-30,18-304,CLE,@,PHO,L (-9),1,41:00,...,10,12,8,1,0,7,1,21,14.7,-3
2,3,3,2003-11-01,18-306,CLE,@,POR,L (-19),1,39:00,...,4,4,6,2,0,2,3,8,5.0,-21
3,4,4,2003-11-05,18-310,CLE,NaN,DEN,L (-4),1,41:00,...,9,11,7,2,3,2,1,7,11.2,-3
4,5,5,2003-11-07,18-312,CLE,@,IND,L (-1),1,44:00,...,5,5,3,0,0,7,2,23,9.0,-7
5,6,6,2003-11-08,18-313,CLE,NaN,WAS,W (+13),1,45:00,...,3,8,9,1,1,2,1,17,15.7,+14
6,7,7,2003-11-10,18-315,CLE,NaN,NYK,W (+14),1,34:00,...,4,5,4,1,1,2,0,17,15.8,+20
7,8,8,2003-11-12,18-317,CLE,@,MIA,L (-5),1,42:00,...,2,3,7,2,1,4,3,18,12.8,-4
8,9,9,2003-11-14,18-319,CLE,@,BOS,L (-9),1,35:00,...,4,5,3,0,0,4,3,10,0.8,-1
9,10,10,2003-11-15,18-320,CLE,NaN,PHI,W (+3),1,47:00,...,5,5,8,1,2,5,2,22,15.6,+8


In [122]:
url = 'https://www.basketball-reference.com/teams/LAL/2019_games.html'
html = requests.get(url).content
df_list = pd.read_html(html)
lakers = df_list[0]

In [123]:
lakers[lakers.G=='G'].index

Int64Index([20, 41, 62, 83], dtype='int64')

In [124]:
lakers.drop(lakers[lakers.G=='G'].index, inplace=True )

In [125]:
lakers.reset_index(drop=True, inplace=True)

In [126]:
lakers

,G,Date,Start (ET),Unnamed: 3,Unnamed: 4,Unnamed: 5,Opponent,Unnamed: 7,Unnamed: 8,Tm,Opp,W,L,Streak,Notes
0,1,"Thu, Oct 18, 2018",10:30p,NaN,Box Score,@,Portland Trail Blazers,L,NaN,119,128,0,1,L 1,NaN
1,2,"Sat, Oct 20, 2018",10:30p,NaN,Box Score,NaN,Houston Rockets,L,NaN,115,124,0,2,L 2,NaN
2,3,"Mon, Oct 22, 2018",10:30p,NaN,Box Score,NaN,San Antonio Spurs,L,OT,142,143,0,3,L 3,NaN
3,4,"Wed, Oct 24, 2018",10:00p,NaN,Box Score,@,Phoenix Suns,W,NaN,131,113,1,3,W 1,NaN
4,5,"Thu, Oct 25, 2018",10:30p,NaN,Box Score,NaN,Denver Nuggets,W,NaN,121,114,2,3,W 2,NaN
5,6,"Sat, Oct 27, 2018",8:30p,NaN,Box Score,@,San Antonio Spurs,L,NaN,106,110,2,4,L 1,NaN
6,7,"Mon, Oct 29, 2018",8:00p,NaN,Box Score,@,Minnesota Timberwolves,L,NaN,120,124,2,5,L 2,NaN
7,8,"Wed, Oct 31, 2018",10:30p,NaN,Box Score,NaN,Dallas Mavericks,W,NaN,114,113,3,5,W 1,NaN
8,9,"Sat, Nov 3, 2018",10:00p,NaN,Box Score,@,Portland Trail Blazers,W,NaN,114,110,4,5,W 2,NaN
9,10,"Sun, Nov 4, 2018",9:30p,NaN,Box Score,NaN,Toronto Raptors,L,NaN,107,121,4,6,L 1,NaN


    G -- Games
    Start (ET) -- Time zone = ET
    Tm -- Points
    Opp -- Opponent Points
    W -- Wins
    L -- Losses

In [192]:
years = [2019,2018,2017,2016,2015]
seasons = {}
for year in years:
    seasons[year]={}
    for name, add in teams_d.items():
        url = 'https://www.basketball-reference.com/teams/' + add + '/' + str(year) + '_games.html'
        html = requests.get(url).content
        df_list = pd.read_html(html)
        df = df_list[0]
        df.drop(df[df.G=='G'].index, inplace=True)
        df.reset_index(drop=True, inplace=True)
        df['Opponent'] = df.Opponent.map(lambda x: '_'.join(x.split(' ')))
        df['Date'] = df.Date.map(lambda x: ''.join(str(pd.to_datetime(x)).split(' ')[0].split('-')))
        df.drop(df.columns[[3, 4]], axis=1, inplace=True)
        df.rename(columns = {'Unnamed: 5':'Guest','Unnamed: 7':'Win_or_Lose','Unnamed: 8':'OverTime'}, inplace=True)
        seasons[year][name] = df

In [193]:
seasons[2019]['Los_Angeles_Lakers']

,G,Date,Start (ET),Guest,Opponent,Win_or_Lose,OverTime,Tm,Opp,W,L,Streak,Notes
0,1,20181018,10:30p,@,Portland_Trail_Blazers,L,NaN,119,128,0,1,L 1,NaN
1,2,20181020,10:30p,NaN,Houston_Rockets,L,NaN,115,124,0,2,L 2,NaN
2,3,20181022,10:30p,NaN,San_Antonio_Spurs,L,OT,142,143,0,3,L 3,NaN
3,4,20181024,10:00p,@,Phoenix_Suns,W,NaN,131,113,1,3,W 1,NaN
4,5,20181025,10:30p,NaN,Denver_Nuggets,W,NaN,121,114,2,3,W 2,NaN
5,6,20181027,8:30p,@,San_Antonio_Spurs,L,NaN,106,110,2,4,L 1,NaN
6,7,20181029,8:00p,@,Minnesota_Timberwolves,L,NaN,120,124,2,5,L 2,NaN
7,8,20181031,10:30p,NaN,Dallas_Mavericks,W,NaN,114,113,3,5,W 1,NaN
8,9,20181103,10:00p,@,Portland_Trail_Blazers,W,NaN,114,110,4,5,W 2,NaN
9,10,20181104,9:30p,NaN,Toronto_Raptors,L,NaN,107,121,4,6,L 1,NaN


In [197]:
seasons[2019]['Los_Angeles_Lakers'].iloc[0].Guest=='@'

True

In [204]:
games = {}
for i in range(82):
    team = 'Los_Angeles_Lakers'
    year = 2019
    date = seasons[year][team].iloc[i].Date
    guest = seasons[year][team].iloc[i].Guest=='@'
    opponent = seasons[year][team].iloc[i].Opponent
    if guest:
        url = 'https://www.basketball-reference.com/boxscores/'+str(date)+str(0)+teams_d[opponent]+'.html'
        html = requests.get(url).content
        df_list = pd.read_html(html)
        df1 = df_list[0]
        df2 = df_list[1]
        games[i] = pd.concat([df1, df2], axis=1)
    else:
        url = 'https://www.basketball-reference.com/boxscores/'+str(date)+str(0)+teams_d[team]+'.html'
        html = requests.get(url).content
        df_list = pd.read_html(html)
        df1 = df_list[2]
        df2 = df_list[3]
        games[i] = pd.concat([df1, df2], axis=1)

In [205]:
games[4]

Unnamed: 0_level_0 Basic Box Score Stats                 \
                    Starters                    MP             FG   
0               LeBron James                 34:40             10   
1                 Kyle Kuzma                 34:17              8   
2                 Lonzo Ball                 33:15              5   
3                  Josh Hart                 31:55              3   
4               JaVale McGee                 31:43             10   
5                   Reserves                    MP             FG   
6           Lance Stephenson                 24:45              5   
7   Kentavious Caldwell-Pope                 22:01              2   
8         Johnathan Williams                 12:28              1   
9      Sviatoslav Mykhailiuk                 12:24              2   
10           Michael Beasley                  2:32              0   
11               Ivica Zubac          Did Not Play   Did Not Play   
12               Rajon Rondo         Not With Team  Not With Team   
13            Brandon Ingram         Not With Team  Not With Team   
14               Team Totals                   240             46   

                                                                               \
              FGA            FG%             3P            3PA            3P%   
0              17           .588              1              5           .200   
1              18           .444              2              6           .333   
2              10           .500              2              5           .400   
3              10           .300              1              6           .167   
4              16           .625              0              0            NaN   
5             FGA            FG%             3P            3PA            3P%   
6              11           .455              2              3           .667   
7               5           .400              2              4           .500   
8               2           .500              0              0            NaN   
9               3           .667              0              0            NaN   
10              0            NaN              0              0            NaN   
11   Did Not Play   Did Not Play   Did Not Play   Did Not Play   Did Not Play   
12  Not With Team  Not With Team  Not With Team  Not With Team  Not With Team   
13  Not With Team  Not With Team  Not With Team  Not With Team  Not With Team   
14             92           .500             10             29           .345   

                                  ... Advanced Box Score Stats                 \
               FT            FTA  ...                     ORB%           DRB%   
0               7             10  ...                      0.0           35.4   
1               4              4  ...                      0.0            9.8   
2               0              0  ...                      0.0           20.1   
3               5              6  ...                      0.0           17.5   
4               1              1  ...                     17.6            7.0   
5              FT            FTA  ...                     ORB%           DRB%   
6               0              0  ...                      9.0            9.0   
7               2              2  ...                      0.0           10.1   
8               0              0  ...                      9.0            9.0   
9               0              0  ...                      9.0            9.0   
10              0              0  ...                      0.0            0.0   
11   Did Not Play   Did Not Play  ...            Not With Team  Not With Team   
12  Not With Team  Not With Team  ...            Not With Team  Not With Team   
13  Not With Team  Not With Team  ...             Did Not Play   Did Not Play   
14             19             23  ...                     20.9           76.7   

                                                                               \

In [207]:
seasons[2019]['Los_Angeles_Lakers'].iloc[[4]]

,G,Date,Start (ET),Guest,Opponent,Win_or_Lose,OverTime,Tm,Opp,W,L,Streak,Notes
4,5,20181025,10:30p,NaN,Denver_Nuggets,W,NaN,121,114,2,3,W 2,NaN


In [273]:
schedules = pd.DataFrame()
for year in [2015,2016,2017,2018]:
    for month in ['october', 'november', 'december', 'january', 'february', 'march', 'april', 'may', 'june']:
        url = 'https://www.basketball-reference.com/leagues/NBA_'+ str(year) +'_games-'+ month +'.html'
        html = requests.get(url).content
        df_list = pd.read_html(html)
        df = df_list[0]
        if 'Playoffs' in list(df.Date):
            df.drop(df[df.Date=='Playoffs'].index, inplace=True)
        df.drop(df[df.Date=='Date'].index, inplace=True)
        df.drop(df.columns[[6, 9]], axis=1, inplace=True)
        df['Date'] = df.Date.map(lambda x: ''.join(str(pd.to_datetime(x)).split(' ')[0].split('-')))
        df.rename(columns = {'Unnamed: 7':'OverTime'}, inplace=True)
        df.reset_index(drop=True, inplace=True)
        schedules = schedules.append(df, ignore_index=True, sort=False)

In [274]:
schedules

,Date,Start (ET),Visitor/Neutral,PTS,Home/Neutral,PTS.1,OverTime,Attend.
0,20141028,10:30p,Houston Rockets,108,Los Angeles Lakers,90,NaN,18997
1,20141028,8:00p,Orlando Magic,84,New Orleans Pelicans,101,NaN,17097
2,20141028,8:00p,Dallas Mavericks,100,San Antonio Spurs,101,NaN,19615
3,20141029,7:30p,Brooklyn Nets,105,Boston Celtics,121,NaN,18624
4,20141029,7:00p,Milwaukee Bucks,106,Charlotte Hornets,108,OT,19439
5,20141029,9:00p,Detroit Pistons,79,Denver Nuggets,89,NaN,17136
6,20141029,7:00p,Philadelphia 76ers,91,Indiana Pacers,103,NaN,18165
7,20141029,8:00p,Minnesota Timberwolves,101,Memphis Grizzlies,105,NaN,17731
8,20141029,7:30p,Washington Wizards,95,Miami Heat,107,NaN,19744
9,20141029,8:00p,Chicago Bulls,104,New York Knicks,80,NaN,19812


In [267]:
try:
    url = 'https://www.basketball-reference.com/leagues/NBA_2019_games-june.html'
    html = requests.get(url).content
    df_list = pd.read_html(html)
    df = df_list[0]
    if 'Playoffs' in list(df.Date):
        df.drop(df[df.Date=='Playoffs'].index, inplace=True)
    df.drop(df[df.Date=='Date'].index, inplace=True)
    df.drop(df.columns[[6, 9]], axis=1, inplace=True)
    df['Date'] = df.Date.map(lambda x: ''.join(str(pd.to_datetime(x)).split(' ')[0].split('-')))
    df.rename(columns = {'Unnamed: 7':'OverTime'}, inplace=True)
    df.reset_index(drop=True, inplace=True)
except:
    continue

SyntaxError: 'continue' not properly in loop (<ipython-input-267-2007df88e314>, line 17)

# Player Stats

    Rk -- Rank
    Pos -- Position
    Age -- Age of Player at the start of February 1st of that season.
    Tm -- Team
    G -- Games
    GS -- Games Started
    MP -- Minutes Played Per Game
    FG -- Field Goals Per Game
    FGA -- Field Goal Attempts Per Game
    FG% -- Field Goal Percentage
    3P -- 3-Point Field Goals Per Game
    3PA -- 3-Point Field Goal Attempts Per Game
    3P% -- FG% on 3-Pt FGAs.
    2P -- 2-Point Field Goals Per Game
    2PA -- 2-Point Field Goal Attempts Per Game
    2P% -- FG% on 2-Pt FGAs.
    eFG% -- Effective Field Goal Percentage
    This statistic adjusts for the fact that a 3-point field goal is worth one more point than a 2-point field goal.
    FT -- Free Throws Per Game
    FTA -- Free Throw Attempts Per Game
    FT% -- Free Throw Percentage
    ORB -- Offensive Rebounds Per Game
    DRB -- Defensive Rebounds Per Game
    TRB -- Total Rebounds Per Game
    AST -- Assists Per Game
    STL -- Steals Per Game
    BLK -- Blocks Per Game
    TOV -- Turnovers Per Game
    PF -- Personal Fouls Per Game
    PTS -- Points Per Game

In [28]:
to_num_list = ['Rk', 'Age', 'G', 'GS', 'MP', 'FG', 'FGA', 'FG%', '3P', '3PA', '3P%', '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA', 'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS']

In [29]:
years = [2019,2018,2017,2016,2015]
player_stats = {}
for year in years:
    url = 'https://www.basketball-reference.com/leagues/NBA_'+str(year)+'_per_game.html'
    html = requests.get(url).content
    df_list = pd.read_html(html)
    df = df_list[0]
    df.drop(df[df.Rk=='Rk'].index, inplace=True)
    df[to_num_list] = df[to_num_list].apply(pd.to_numeric)
    player_stats[year] = df

In [30]:
player_stats[2019]

,Rk,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,1,Alex Abrines,SG,25,OKC,31,2,19.0,1.8,5.1,...,0.923,0.2,1.4,1.5,0.6,0.5,0.2,0.5,1.7,5.3
1,2,Quincy Acy,PF,28,PHO,10,0,12.3,0.4,1.8,...,0.700,0.3,2.2,2.5,0.8,0.1,0.4,0.4,2.4,1.7
2,3,Jaylen Adams,PG,22,ATL,34,1,12.6,1.1,3.2,...,0.778,0.3,1.4,1.8,1.9,0.4,0.1,0.8,1.3,3.2
3,4,Steven Adams,C,25,OKC,80,80,33.4,6.0,10.1,...,0.500,4.9,4.6,9.5,1.6,1.5,1.0,1.7,2.6,13.9
4,5,Bam Adebayo,C,21,MIA,82,28,23.3,3.4,5.9,...,0.735,2.0,5.3,7.3,2.2,0.9,0.8,1.5,2.5,8.9
5,6,Deng Adel,SF,21,CLE,19,3,10.2,0.6,1.9,...,1.000,0.2,0.8,1.0,0.3,0.1,0.2,0.3,0.7,1.7
6,7,DeVaughn Akoon-Purcell,SG,25,DEN,7,0,3.1,0.4,1.4,...,0.500,0.1,0.4,0.6,0.9,0.3,0.0,0.3,0.6,1.0
7,8,LaMarcus Aldridge,C,33,SAS,81,81,33.2,8.4,16.3,...,0.847,3.1,6.1,9.2,2.4,0.5,1.3,1.8,2.2,21.3
8,9,Rawle Alkins,SG,21,CHI,10,1,12.0,1.3,3.9,...,0.667,1.1,1.5,2.6,1.3,0.1,0.0,0.8,0.7,3.7
9,10,Grayson Allen,SG,23,UTA,38,2,10.9,1.8,4.7,...,0.750,0.1,0.5,0.6,0.7,0.2,0.2,0.9,1.2,5.6


In [32]:
player_stats[2019].sort_values(by=['PTS'], inplace=True, ascending=False)

In [34]:
player_stats[2019].head(30)

,Rk,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
277,206,James Harden,PG,29,HOU,78,78,36.8,10.8,24.5,...,0.879,0.8,5.8,6.6,7.5,2.0,0.7,5.0,3.1,36.1
249,183,Paul George,SF,28,OKC,77,77,36.9,9.2,21.0,...,0.839,1.4,6.8,8.2,4.1,2.2,0.4,2.7,2.8,28.0
19,18,Giannis Antetokounmpo,PF,24,MIL,72,72,32.8,10.0,17.3,...,0.729,2.2,10.3,12.5,5.9,1.3,1.5,3.7,3.2,27.7
207,155,Joel Embiid,C,24,PHI,64,64,33.7,9.1,18.7,...,0.804,2.5,11.1,13.6,3.7,0.7,1.9,3.5,3.3,27.5
344,258,LeBron James,SF,34,LAL,55,55,35.2,10.1,19.9,...,0.665,1.0,7.4,8.5,8.3,1.3,0.6,3.6,1.7,27.4
165,124,Stephen Curry,PG,30,GSW,69,69,33.8,9.2,19.4,...,0.916,0.7,4.7,5.3,5.2,1.3,0.4,2.8,2.4,27.3
71,61,Devin Booker,SG,22,PHO,64,64,35.0,9.2,19.6,...,0.866,0.6,3.5,4.1,6.8,0.9,0.2,4.1,3.1,26.6
412,301,Kawhi Leonard,SF,27,TOR,60,60,34.0,9.3,18.8,...,0.854,1.3,6.0,7.3,3.3,1.8,0.4,2.0,1.5,26.6
198,150,Kevin Durant,PF,30,GSW,78,78,34.6,9.2,17.7,...,0.885,0.4,5.9,6.4,5.9,0.7,1.1,2.9,2.0,26.0
167,126,Anthony Davis,C,25,NOP,56,56,33.0,9.5,18.3,...,0.794,3.1,8.9,12.0,3.9,1.6,2.4,2.0,2.4,25.9


# MVP Voting

    Age -- Age of Player at the start of February 1st of that season.
    Tm -- Team
    Voting
    First -- First Place Votes
    Totals
    G -- Games
    Per Game
    MP -- Minutes Played Per Game
    PTS -- Points Per Game
    TRB -- Total Rebounds Per Game
    AST -- Assists Per Game
    STL -- Steals Per Game
    BLK -- Blocks Per Game
    Shooting
    FG% -- Field Goal Percentage
    3P% -- 3-Point Field Goal Percentage
    FT% -- Free Throw Percentage
    Advanced
    WS -- Win Shares
    An estimate of the number of wins contributed by a player.
    WS/48 -- Win Shares Per 48 Minutes
    An estimate of the number of wins contributed by a player per 48 minutes (league average is approximately .100)

In [52]:
to_num_list = ['Age', 'First', 'Pts Won', 'Pts Max', 'Share', 'G', 'MP', 'PTS', 'TRB', 'AST', 'STL', 'BLK', 'FG%', '3P%', 'FT%', 'WS', 'WS/48']


In [53]:
years = [2018,2017,2016,2015,2014,2013,2012,2011,2010,2009,2008]
MVP_voting = {}
for year in years:
    url = 'https://www.basketball-reference.com/awards/awards_'+str(year)+'.html'
    html = requests.get(url).content
    df_list = pd.read_html(html)
    df = df_list[0]
    df.columns = df.columns.droplevel()
    df[to_num_list] = df[to_num_list].apply(pd.to_numeric)
    MVP_voting[year] = df

In [56]:
MVP_voting[2016]

,Rank,Player,Age,Tm,First,Pts Won,Pts Max,Share,G,MP,PTS,TRB,AST,STL,BLK,FG%,3P%,FT%,WS,WS/48
0,1,Stephen Curry,27,GSW,131.0,1310.0,1310,1.000,79,34.2,30.1,5.4,6.7,2.1,0.2,0.504,0.454,0.908,17.9,0.318
1,2,Kawhi Leonard,24,SAS,0.0,634.0,1310,0.484,72,33.1,21.2,6.8,2.6,1.8,1.0,0.506,0.443,0.874,13.7,0.277
2,3,LeBron James,31,CLE,0.0,631.0,1310,0.482,76,35.6,25.3,7.4,6.8,1.4,0.6,0.520,0.309,0.731,13.6,0.242
3,4,Russell Westbrook,27,OKC,0.0,486.0,1310,0.371,80,34.4,23.5,7.8,10.4,2.0,0.3,0.454,0.296,0.812,14.0,0.245
4,5,Kevin Durant,27,OKC,0.0,147.0,1310,0.112,72,35.8,28.2,8.2,5.0,1.0,1.2,0.505,0.387,0.898,14.5,0.270
5,6,Chris Paul,30,LAC,0.0,107.0,1310,0.082,74,32.7,19.5,4.2,10.0,2.1,0.2,0.462,0.371,0.896,12.7,0.253
6,7,Draymond Green,25,GSW,0.0,50.0,1310,0.038,81,34.7,14.0,9.5,7.4,1.5,1.4,0.490,0.388,0.696,11.1,0.190
7,8,Damian Lillard,25,POR,0.0,26.0,1310,0.020,75,35.7,25.1,4.0,6.8,0.9,0.4,0.419,0.375,0.892,9.2,0.165
8,9,James Harden,26,HOU,0.0,9.0,1310,0.007,82,38.1,29.0,6.1,7.5,1.7,0.6,0.439,0.359,0.860,13.3,0.204
9,10,Kyle Lowry,29,TOR,0.0,6.0,1310,0.005,77,37.0,21.2,4.7,6.4,2.1,0.4,0.427,0.388,0.811,11.6,0.196
